In [ ]:
%load_ext autoreload
%autoreload 2

: 

In [3]:
import transformers
from transformers import LayoutLMForTokenClassification,\
    LayoutLMTokenizer, AdamW, LayoutLMv2Processor, LayoutLMv2ForTokenClassification
from tensordict import TensorDict
from datasets import load_dataset
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import ImageDraw, Image
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import logging
from torchvision.transforms import PILToTensor

import os 
if 'notebooks' in os.getcwd():
    os.chdir("..")

ModuleNotFoundError: No module named 'transformers'

In [ ]:
logging.basicConfig(filename='logs/train.log', encoding='utf-8', level=logging.DEBUG)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

In [5]:
# dataset = load_dataset("katanaml/cord", )

dataset = load_dataset("darentang/sroie")

## Creating PyTorch Datasets, DataLoader

In [ ]:
processor =LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")


In [ ]:
class SROIE(Dataset):
    def __init__(self, 
                 data,
                 encode : bool = False) -> None:
        super().__init__()

        if encode:
            self.X = []
            for example in tqdm(data):
                X= SROIE.encode(example)
                self.X.append(X)

        else:
            self.X = data

    @staticmethod
    def encode(
        example, 
        max_seq_length = 512, 
        pad_token_box = [0,0,0,0]
    ):
        words = example['words']
        boxes = example['bboxes']
        image = Image.open(example['image_path'])
        word_labels = example['ner_tags']

        encoded_inputs = processor(
            image, 
            words, 
            boxes=boxes, 
            word_labels=word_labels, 
            padding="max_length", 
            truncation=True, 
            return_tensors="pt"
        )

        # token_boxes = []
        # for word, box in zip(words, bboxes):
        #     word_tokens = tokenizer.tokenize(word)
        #     token_boxes.extend([box] * len(word_tokens))

        
        # special_tokens_count = 2 
        # if len(token_boxes) > max_seq_length - special_tokens_count:
        #     token_boxes = token_boxes[: (max_seq_length - special_tokens_count)]

        # # add bounding boxes of cls + sep tokens
        # token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]

        # encoding = tokenizer(' '.join(words), padding='max_length', truncation=True)
        # # Padding of token_boxes up the bounding boxes to the sequence length.
        # input_ids = torch.tensor(tokenizer(' '.join(words), truncation=True)["input_ids"])
        # padding_length = max_seq_length - len(input_ids)
        # token_boxes += [pad_token_box] * padding_length

        # encoding['labels'] = torch.tensor(example['ner_tags'])
        # encoding['bbox'] = torch.tensor(token_boxes)
        # # encoding['input_ids'] = torch.tensor(encoding['input_ids'])
        # encoding['token_type_ids'] = torch.tensor(encoding['token_type_ids'])
        # encoding['attention_mask'] = torch.tensor(encoding['attention_mask'])
        
        return encoded_inputs
    
    def __getitem__(self, index: int):
        return self.X[index]

    def __len__(self):
        return len(self.X)

In [ ]:
data = []
for example in tqdm(dataset['train']):
    words = example['words']
    boxes = example['bboxes']
    image = Image.open(example['image_path'])
    word_labels = example['ner_tags']

    try:
        encoded_inputs = processor(
            image, 
            words, 
            boxes=boxes, 
            word_labels=word_labels, 
            padding="max_length", 
            truncation=True, 
            return_tensors="pt"
        )

        data.append(encoded_inputs)
    except:
        pass

In [ ]:
dataloader = DataLoader(
    df,
    shuffle=True,
    batch_size= 1
)

In [17]:
example = dataset['train'][0]

In [18]:
words = example['words']

## Importing model

In [6]:
n_labels = 8
model = LayoutLMv2ForTokenClassification.from_pretrained(
    'microsoft/layoutlmv2-base-uncased',
    num_labels=n_labels
)
model.to(device)

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LayoutLMForTokenClassification(
  (layoutlm): LayoutLMModel(
    (embeddings): LayoutLMEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (x_position_embeddings): Embedding(1024, 768)
      (y_position_embeddings): Embedding(1024, 768)
      (h_position_embeddings): Embedding(1024, 768)
      (w_position_embeddings): Embedding(1024, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LayoutLMEncoder(
      (layer): ModuleList(
        (0-11): 12 x LayoutLMLayer(
          (attention): LayoutLMAttention(
            (self): LayoutLMSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
 

### Number of treinable / non-treinable parameters

In [7]:
print(f'''
    Model Info
    -----------------
    
    Treinable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}
    Non Treinable params: {sum(p.numel() for p in model.parameters() if not p.requires_grad)}

''')


    Model Info
    -----------------
    
    Treinable params: 112634120
    Non Treinable params: 0




In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

global_step = 0
num_train_epochs = 4
t_total = len(dataloader) * num_train_epochs # total number of training steps 

#put the model in training mode
model.train()
for epoch in tqdm(range(num_train_epochs)):
  logging.info(f"Epoch: {epoch}")
  running_loss = 0.0
  correct = 0
  for X in dataloader:
      input_ids = X["input_ids"].to(device)
      bbox = X["bbox"].to(device)
      attention_mask = X["attention_mask"].to(device)
      token_type_ids = X["token_type_ids"].to(device)
      labels = X["labels"].to(device)
      image = X["image"].to(device)

      # forward pass
      outputs = model(
        input_ids=input_ids, 
        bbox=bbox, 
        attention_mask=attention_mask, 
        token_type_ids=token_type_ids,
        image= image,
        labels=labels
      )
      loss = outputs.loss

      running_loss += loss.item()
      predictions = outputs.logits.argmax(-1)
      correct += (predictions == labels).float().sum()

      # backward pass to get the gradients 
      loss.backward()

      # update
      optimizer.step()
      optimizer.zero_grad()
      global_step += 1
  
  logging.info("Loss:", running_loss / X["input_ids"].shape[0])
  accuracy = 100 * correct / len(data)
  logging.info("Training accuracy:", accuracy.item())